In [1]:
import numpy as np
import numpy.random as rd
import os
import scipy.stats as st
import itertools

In [2]:
os.chdir("..")
home_dir = os.getcwd()
os.chdir("src")
from sdle import SDLE
from sdem import SDEM
from smartsifter import SmartSifter
os.chdir(home_dir)

## データの生成

In [3]:
seed = 0
#データ生成(離散値ベクトル)
rd.seed(seed)
T = 500 #データ数(観測数)
x_1 = np.array([np.argmax(i) for i in rd.multinomial(1, [1/6]*6,T)])
x_1 = np.array([np.argmax(i) for i in rd.multinomial(1, [1]*1,T)])
x = x_1.copy()
#x_2 = rd.binomial(1, 0.5, T)
#x = np.c_[x_1, x_2]

#データ生成(連続変数ベクトル)
n = [200, 150, 150] #各データ数
K = 3 #潜在変数の数
D = 2 #次元

#mu:D次元
mu_true = np.array(
    [[0.2, 0.5],
     [1.2, 0.5],
     [2.0, 0.5]])

#sigma: D×D次元
sigma_true = np.array(
    [[[0.1,  0.085], [0.085, 0.1]],
     [[0.1, -0.085], [-0.085, 0.1]],
     [[0.1,  0.085], [0.085, 0.1]]
    ])

rd.seed(seed)
org_data = None
for i in range(K):
    #k_0 に属するデータを生成
    if org_data is None:
        org_data = np.c_[st.multivariate_normal.rvs(mean=mu_true[i], cov=sigma_true[i], size=n[i]), np.ones(n[i])*i]
        
    #k_1, k_2に属するデータを生成し、結合する
    else:
        tmp_data = np.c_[st.multivariate_normal.rvs(mean=mu_true[i], cov=sigma_true[i], size=n[i]), np.ones(n[i])*i]
        org_data = np.r_[org_data, tmp_data]

#print(org_data)
y = org_data[:, :2]

## パラメータの設定

In [4]:
#パラメータの設定
#SDLE
A = list(itertools.product(set(x)))
#A = list(itertools.product(set(x_1), set(x_2)))
r = 1 / len(x) #忘却パラメータ
beta = 1 #正の定数

#SDEM
alpha = 1.0 #(1.0~2.0)
r = 1 / T
k = K #潜在変数の数
d = D #次元
#SmartSifter
r_h = 0.1

In [5]:
ss = SmartSifter(r, beta, A, alpha, k, r_h, SDEM, SDLE)
ss.train(x, y) 

/Users/satetsu/program/python/datamining/anomaly_detection_with_data_mining/src/smartsifter.py:73: RuntimeWarning: divide by zero encountered in log
  s_l = -np.log(p_prev_params) #シャノン情報量を計算


In [6]:
ss.S_L

array([         inf,   2.58544416,   3.6754927 ,   3.57323642,
         3.47336257,   3.80565606,   4.1772957 ,   4.20398507,
         4.82100441,   4.38155106,   4.25303319,   4.80255147,
         5.69977144,   4.66198178,   5.4331336 ,   4.84640531,
         4.65630616,   4.82478645,   5.49775745,   4.91420681,
         4.8512431 ,   4.95678749,   5.0278169 ,   5.00334855,
         5.00985813,   5.10430662,   5.18111058,   5.35672816,
         5.41541181,   5.26822829,   5.29368184,   5.29750877,
         5.5616598 ,   5.32815003,   5.38477844,   5.86314   ,
         6.06361961,   5.77003792,   5.48347895,   5.61735907,
         5.53633637,   5.88601053,   6.4715083 ,   6.25215651,
         5.63913781,   5.76428069,   5.93838758,   5.99852098,
         5.95002437,   5.95056835,   6.78264444,   5.76014327,
         5.82555524,   5.86275236,   6.93465881,   6.887704  ,
         5.92036775,   5.99123685,   6.40088378,   6.29995577,
         6.18960764,   6.22848382,   5.96185412,   6.08